In [ ]:
# Set global utool flags
import utool as ut
ut.util_io.__PRINT_WRITES__ = False
ut.util_io.__PRINT_READS__ = False
ut.util_parallel.__FORCE_SERIAL__ = True
ut.util_cache.VERBOSE_CACHE = False
ut.NOT_QUIET = False

# Matplotlib stuff
%pylab inline
%load_ext autoreload
%autoreload

# Define database for this test run
import ibeis
db = 'PZ_Master1'
#db = 'GZ_ALL'
#db = 'NNP_MasterGIRM_core'
 
# Setup database specific configs
if db == 'PZ_MTEST' or db == 'GZ_ALL':
    seperability_annot_cfgs = ['ctrl']
else:
    seperability_annot_cfgs = ['timequalctrl']

if db == 'PZ_Master1':
    varysize = ['varysize_pzm']
    varypername = ['varypername_pzm']
elif db == 'GZ_ALL':
    varysize = ['varysize_gz']
    varypername = ['varypername_gz']
elif db == 'NNP_MasterGIRM_core':
    varysize = ['varysize_girm']
    varypername = ['varypername_girm']

# Baseline Experiments

In [ ]:
test_result = ibeis.run_experiment(
    e='rank_cdf', 
    db=db, 
    a=['unctrl', 'ctrl::unctrl:qpername=1,qview_ext=0'],
    t=['baseline'])
#test_result.print_unique_annot_config_stats()
_ = test_result.draw_func()

In [ ]:
# Baseline Timedeltas FP
test_result = ibeis.run_experiment(
    e='timedelta_hist',
    db=db, 
    a=['ctrl::unctrl:qpername=1,qview_ext=0'], 
    t=['baseline'],
    truepos=True)
test_result.draw_func()

In [ ]:
# Baseline Timedeltas FP
test_result = ibeis.run_experiment(
    e='timedelta_hist',
    db=db, 
    a=['ctrl::unctrl:qpername=1,qview_ext=0'], 
    t=['baseline'],
    falsepos=True)
test_result.draw_func()

In [ ]:
# Time Experiment 
test_result = ibeis.run_experiment(
    e='rank_cdf',
    db=db, 
    a=['largetimedelta', 'ctrl'], 
    t=['baseline'])
test_result.draw_func()

In [ ]:
# Timedelta PIE for new False Pos 
test_result = ibeis.run_experiment(
    e='timedelta_hist',
    db=db, 
    a=['largetimedelta'], 
    t=['baseline'],
    falsepos=True)
test_result.draw_func()

# Database Size Experiments

In [ ]:
# K Experiment 
test_result = ibeis.run_experiment(
    e='rank_surface',
    db=db, 
    a=varysize, 
    t=['candidacy_k'])
#test_result.print_unique_annot_config_stats()
test_result.draw_func()

# Scoring Mechanism Experiments

In [ ]:
# Namescore Experiment
test_result = ibeis.run_experiment(
    e='rank_cdf',
    db=db, 
    a=varypername, 
    t=['candidacy_namescore'])
test_result.draw_func()

# Viewpoint Experiments

In [ ]:
try:
    test_result = ibeis.run_experiment(
        e='rank_cdf',
        db=db, 
        a=['viewpoint_compare'], 
        t=['baseline'])
    test_result.draw_func()
except AssertionError as ex:
    ut.printex(ex, 'Database does not support this test')

# Invariance Experiments

In [ ]:
test_result = ibeis.run_experiment(
    e='rank_cdf',
    db=db, 
    a=['ctrl'], 
    t=['candidacy_invariance'])
test_result.draw_func()

In [ ]:
# Invariance + Viewpoint Experiment
try:
    test_result = ibeis.run_experiment(
        e='rank_cdf',
        db=db, 
        a=['viewpoint_compare'], 
        t=['candidacy_invariance'], 
        test_cfgx_slice=slice(6, None))
    test_result.draw_func()
except AssertionError as ex:
    ut.printex(ex, 'Database does not support this test')

In [ ]:
# Invariance + Time Experiment
test_result = ibeis.run_experiment(
    e='rank_cdf',
    db=db, 
    a=['timectrl'], 
    t=['baseline:AQH=False,AI=True',
       'baseline:AQH=True,AI=False',
       'baseline:AQH=False,AI=False'])
_ = test_result.draw_func()

In [ ]:
# Invariance + Time Experiment
test_result = ibeis.run_experiment(
    e='rank_cdf',
    db=db, 
    a=seperability_annot_cfgs, 
    t=['baseline:AQH=False,AI=True',
       'baseline:AQH=True,AI=False',
       'baseline:AQH=False,AI=False'])
_ = test_result.draw_func()

# Score Seperability Experiments

## Success Cases

In [ ]:
# Score Seperability of Good Cases
test_result = ibeis.run_experiment(
    e='scores',
    db=db, 
    a=seperability_annot_cfgs, 
    t=['invarbest'],
    f=[':fail=False,min_gf_timedelta=None'],
)
_ = test_result.draw_func()

## All Cases

In [ ]:
# Score Seperability with NO FILTERING
test_result = ibeis.run_experiment(
    e='scores',
    db=db, 
    a=seperability_annot_cfgs, 
    t=['invarbest'],
    f=[':fail=None,min_gf_timedelta=None']
)
_ = test_result.draw_func()

In [ ]:
test_result = ibeis.run_experiment(
    e='taghist',
    db=db, 
    a=seperability_annot_cfgs, 
    t=['invarbest'],
    f=[':fail=None,min_gf_timedelta=None']
)
_ = test_result.draw_func()

## Success cases with large timedelta for groundfalse cases

In [ ]:
# Score Seperability of Good Cases where large timedelta gf are gone
test_result = ibeis.run_experiment(
    e='scores',
    db=db, 
    a=seperability_annot_cfgs, 
    t=['invarbest'],
    f=[':fail=False,min_gf_timedelta=12h']
)
_ = test_result.draw_func()

## All cases with large timedelta for groundfalse cases

In [ ]:
# Score Seperability of 
test_result = ibeis.run_experiment(
    e='scores',
    db=db, 
    a=seperability_annot_cfgs, 
    t=['baseline:AQH=True,AI=False'],
    f=[':fail=None,min_gf_timedelta=12h']
) 
_ = test_result.draw_func()

In [ ]:
# Investigate the reasons for the high scoring groundfalse cases
test_result = ibeis.run_experiment(
    e='taghist',
    db=db, 
    a=['timecontrolled'], 
    t=['invarbest'],
    f=[':fail=None,min_gf_timedelta=12h']
)
_ = test_result.draw_func()